# Segmenting and Clustering Neighborhoods in Toronto

## Part 1

This notebook contains the procedures involved in the first part of the assignment. Before we do anything else, we must import the necessary packages.

In [1]:
import sys
!{sys.executable} -m pip install geopy folium
import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
Libraries imported.


### Scrape Wikipedia page

Now we can begin the assignment. The first step will be to collect the Toronto neighborhood data from a Wikipedia page. We will do this using the read_html function in pandas. Since this function will return our data as a list of data frames, we will also have to unite them all into a single data frame using the concatenate function. 

In [2]:
import pandas as pd
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url) 
df = pd.concat(dfs)
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df.head()

,Postcode,Borough,Neighbourhood,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,M1A,Not assigned,Not assigned,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M2A,Not assigned,Not assigned,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M3A,North York,Parkwoods,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M4A,North York,Victoria Village,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,M5A,Downtown Toronto,Harbourfront,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Remove unassigned postal codes

As you can see, in the last step, we obtained a dataframe with the information from the Wikipedia page. The data we need are actually in the final three columns of this data frame, and we will therefore select them and view them in the following step. We will also take the opportunity to remove rows where a post code has not been assigned, as well as rows containing 'NaN' values.

In [3]:
df = df[['Borough','Neighbourhood','Postcode']]
to_drop = ['Not assigned']
df = df[~df['Borough'].isin(to_drop)]
df.head()

,Borough,Neighbourhood,Postcode
3,North York,Parkwoods,M3A
4,North York,Victoria Village,M4A
5,Downtown Toronto,Harbourfront,M5A
6,North York,Lawrence Heights,M6A
7,North York,Lawrence Manor,M6A


### Group neighborhoods by postal code

Our next step is to group all neighborhoods which share the same postal code. We will do this using the groupby function, joining rows by postal code and borough. We will also take the opportunity to rename the postal code column, so it looks like the template in the task instructions.

In [4]:
df2=df.groupby("Postcode").agg(lambda x:','.join(set(x)))
df2 = df2.drop([df2.index[0] , df2.index[104], df2.index[105]])
df2.reset_index(level=0, inplace=True)
df2.rename(columns={"Postcode": "PostalCode", "Neighbourhood":"Neighborhood"}, inplace=True)
df2.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Morningside,Guildwood,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Finding the shape of the final data frame

The data frame now looks like the example provided. The final step is to use the shape function to look at the dimensions of our dataframe.

In [5]:
df2.shape

(103, 3)

You have reached the end of Part 1. Now we will go on to Part 2.

## Part 2

Our first step will be to get the latitude and longitude information for each postal code from a CSV file we will download from the task instructions.

In [6]:
url="https://cocl.us/Geospatial_data"
coords=pd.read_csv(url)
coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Our next step will be to join both dataframes using the Postal Code column as the key, in order to obtain a single data frame with information about neighborhoods and location.

In [7]:
neighborhoods = pd.merge(df2, coords, left_on='PostalCode',right_on='Postal Code',validate='many_to_one')
neighborhoods.drop(columns=['Postal Code'], inplace=True)
neighborhoods.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,Guildwood,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Ionview,Kennedy Park,East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile,Clairlea,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside,Scarborough Village West,Cliffcrest",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West,Birch Cliff",43.692657,-79.264848


## Part 3

Now we are ready to learn more about each neighborhood, in order to later cluster them. Our first step will be to find the coordinates of the city of Toronto, in order to build our map.

In [8]:
address = 'Toronto, ON'

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Toronto are 43.653963, -79.387207.


### Plotting the neighborhoods on a map

Using the information we just collected, together with our previous data on the coordinates of each neighborhoood, we will build a map that contains markers for each set of neighborhoods which share a postal code.

In [9]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Collecting information about venues in each neighborhood

Now we will define a function that will get information for venues in each of our neighborhoods. This will be done through the Foursquare API.

In [10]:
# @hidden_cell
CLIENT_ID = 'IHGBJCBT31RG1UETAVMPTBVUXVN4MFG5UTM0BBTO0NPCRL50' # your Foursquare ID
CLIENT_SECRET = '0IGTFFK55WAAI4FGSYVC3YMH121RT3GGSVXG2LRQK5O3W0FO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

We will apply the function and give it some additional instructions, in order to create a dataframe containing information about venues in each of the neighborhoods we have found.

In [13]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

LIMIT = 100

toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )
toronto_venues

Latitude and longitude values of Malvern,Rouge are 43.806686299999996, -79.19435340000001.
Malvern,Rouge
Highland Creek,Rouge Hill,Port Union
Morningside,Guildwood,West Hill
Woburn
Cedarbrae
Scarborough Village
Ionview,Kennedy Park,East Birchmount Park
Golden Mile,Clairlea,Oakridge
Cliffside,Scarborough Village West,Cliffcrest
Cliffside West,Birch Cliff
Scarborough Town Centre,Wexford Heights,Dorset Park
Wexford,Maryvale
Agincourt
Sullivan,Clarks Corners,Tam O'Shanter
Agincourt North,Steeles East,L'Amoreaux East,Milliken
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Oriole,Henry Farm
Bayview Village
Silver Hills,York Mills
Willowdale,Newtonbrook
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South,Flemingdon Park
Downsview North,Wilson Heights,Bathurst Manor
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beac

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern,Rouge",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Morningside,Guildwood,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Morningside,Guildwood,West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa
5,"Morningside,Guildwood,West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
6,"Morningside,Guildwood,West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
7,"Morningside,Guildwood,West Hill",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
8,"Morningside,Guildwood,West Hill",43.763573,-79.188711,Lawrence Ave E & Kingston Rd,43.767704,-79.189490,Intersection
9,"Morningside,Guildwood,West Hill",43.763573,-79.188711,Eggsmart,43.767800,-79.190466,Breakfast Spot


Let's look at how many types of venue we found in total:

In [14]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 270 unique categories.


Our next step is to use one-hot encoding to obtain a clearer description of the types of venues our neighborhoods.

In [15]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head(10)

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,Richmond,King",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.020000,...,0.020000,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.01,0.000000
1,Agincourt,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
2,"Agincourt North,Steeles East,L'Amoreaux East,M...",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
3,"Alderwood,Long Branch",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
4,Bayview Village,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
5,"Bedford Park,Lawrence Manor East",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.041667,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
6,Berczy Park,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.017857,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
7,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.055556
8,"CFB Toronto,Downsview East",0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
9,Caledonia-Fairbanks,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.25,0.000000


Now, let us create a function that will sort the frequency of the venues. 

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

We can now use our function to obtain the 10 most common venue types for each postal code location.

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,Richmond,King",Coffee Shop,Restaurant,Café,Thai Restaurant,Bar,Sushi Restaurant,Concert Hall,Breakfast Spot,Gastropub,Vegetarian / Vegan Restaurant
1,Agincourt,Lounge,Breakfast Spot,Latin American Restaurant,Chinese Restaurant,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,"Agincourt North,Steeles East,L'Amoreaux East,M...",Playground,Park,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
3,"Alderwood,Long Branch",Pizza Place,Gym,Coffee Shop,Pub,Athletics & Sports,Sandwich Place,Pool,Pharmacy,Skating Rink,Dumpling Restaurant
4,Bayview Village,Chinese Restaurant,Bank,Café,Japanese Restaurant,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore


### Clustering the neighborhoods

First we will cluster the neighborhoods into five different types according to the top 10 venues in each location.

In [33]:
# set number of clusters
kclusters = 14

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:30] 

array([1, 1, 2, 8, 1, 1, 1, 1, 2, 2, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 8, 1, 1, 6, 3], dtype=int32)

Now we will create a dataframe to show us our clusters.

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'ClusterLabels', kmeans.labels_)

toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head(10) # check the last columns!

Before we go on to the next step, we will have to conver the 'Cluster Labels' column to integer rather than floats. Otherwise, we will get an error when constructing the map. We will also remove any rows with NaN values which could interfere with the mapping part of our task.

In [ ]:
toronto_merged = toronto_merged.dropna(axis = 0, how ='any').astype({'ClusterLabels': 'int64'})

### Final Result!

For the last step, let us map out the resulting clusters.

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Thank you for following along.